In [1]:
import openai
import json
import pandas as pd

In [2]:
## Set your open ai api key here or store it an environmental variable
#openai.api_key=''

In [3]:
def get_model_completion(messages, functions = None):
    if functions == None:
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo-0613",
            messages = messages
        )
    else:
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo-0613",
            messages = messages,
            functions = functions,
            function_call = "auto"
        )
    message = response["choices"][0]["message"]
    if message.get("function_call"):
        return json.loads(message["function_call"]["arguments"])
    else:
        return message["content"]

# Using the model to create a series of fake product reviews

In [4]:
product_reviews = []
for i in range(10):
    if i % 3 == 0:
        sent = "neutral"
    elif i % 2 == 0:
        sent = "positive"
    else:
        sent = "negative"
    
    prompt = f"""
    You are a product reviewing assistant. You task is to create a product review with a {sent} sentiment
    """
    messages = [
        {
            "role" : "user",
            "content" : prompt
        }
    ]
    review = get_model_completion(messages)
    product_reviews.append(review)

In [5]:
print(product_reviews[0])

Product Review: XYZ Wireless Headphones

I recently got my hands on the XYZ wireless headphones and had the opportunity to test them out for a few weeks. Overall, my experience with these headphones has been quite average, neither being outstanding nor disappointing.

Starting with the design, the XYZ wireless headphones have a sleek and modern look. They are lightweight and comfortable to wear for extended periods, thanks to the cushioned ear cups. However, I found the adjustment mechanism a bit flimsy, as the headphones tended to slip off my head while moving.

In terms of audio quality, the XYZ wireless headphones deliver a decent sound experience. The bass is punchy, and the overall sound is clear and balanced. However, I did notice a slight distortion at higher volumes, which can be a letdown for audio enthusiasts.

One of the standout features of these headphones is the Bluetooth connectivity. Pairing them with my device was a breeze, and the range was sufficient for my needs. Th

# Create a function that would mimic a database write

In [6]:
def write_to_database(product, product_category, sentiment, summary):
    data = {
        "product" : product,
        "product_category" : product_category,
        "sentiment" : sentiment,
        "summary" : summary
    }
    df = pd.DataFrame.from_dict(data, orient = 'index')
    df = df.T
    return df

In [7]:
functions = [
    {
        "name" : "write_to_database",
        "description" : "The function will take the parsed product review and insert it in to the database",
        "parameters" : {
            "type" : "object",
            "properties" : {
                "product" : {
                    "type" : "string",
                    "description" : "The name of the product being reviewed"
                },
                "product_category" : {
                    "type" : "string",
                    "description" : "The product category of the product being reveiwed example: Headphones"
                },
                "sentiment" : {
                    "type" : "string",
                    "enum" : ["negative", "neutral", "positive"]
                },
                "summary" : {
                    "type" : "string",
                    "description" : """This is a very short summary of the view. To be used in a dashboard at the /
                    executive level"""
                }
            },
            "required" : ["product", "product_category", "sentiment", "summary"]
        }
    }
]

In [8]:
for idx, review in enumerate(product_reviews):
    prompt = f"""
    You are a review processing assistant. Your role is to take a review delimited by triple back ticks /
    and return a structured response as defined by the function write_to_database
    ```
    {review}
    ```
    """
    messages = [
        {
            "role" : "user",
            "content" : prompt
        }
    ]
    function_params = get_model_completion(messages, functions = functions)
    if idx == 0:
        df = write_to_database(**function_params)
    else:
        d = write_to_database(**function_params)
        df = pd.concat([df, d], axis = 0).reset_index(drop = True)

In [9]:
df

,product,product_category,sentiment,summary
0,XYZ Wireless Headphones,Headphones,neutral,"Decent sound quality and comfortable design, b..."
1,XYZ Bluetooth Headphones,Headphones,negative,"Disappointing sound quality, poor build, unrel..."
2,Ultimate Wireless Earbuds,Headphones,positive,The Ultimate Wireless Earbuds exceeded my expe...
3,XYZ Bluetooth Earphones,Bluetooth Earphones,positive,"Decent option with satisfactory sound quality,..."
4,Bose QuietComfort 35 II Wireless Headphones,Headphones,positive,The Bose QuietComfort 35 II wireless headphone...
5,XYZ Electric Razor,Electric Razors,negative,"Disappointing performance, cheap build quality..."
6,XYZ Bluetooth Speaker,Bluetooth Speaker,neutral,The XYZ Bluetooth Speaker offers good sound qu...
7,XYZ Bluetooth Earbuds,Bluetooth Earbuds,negative,"Subpar sound quality, short battery life, poor..."
8,Apple AirPods Pro,Headphones,positive,The Apple AirPods Pro are a remarkable product...
9,ABC Grill Master 3000,Grill,neutral,The ABC Grill Master 3000 offers a decent gril...
